# Layoffs Worldwide 2020-2022

This is my second portfolio project in Python :)

---

> This dataset was scraped from **Layoffs.fyi** with the hope to enable Kaggle community to look into analyzing recent mass layoffs and discover useful insights and patterns.

> Original dataset can be tracked at https://layoffs.fyi/

> Credits: **Roger Lee**

---

I'll be joining the above dataset with the free version of **World Cities Database** dataset, available [here](https://simplemaps.com/data/world-cities), to be able to graph cities across the world more precisely on a map.

As I did some exploration of the data, I realized that the dataset is quite imperfect, and isn't representative world-wide. Most of the data contained within it is about layoffs in US-based companies. The disproportion between total number of layoffs in the US vs the rest of the world doesn't match reality, I'm pretty sure.

---

### Sills showcased:


1) Data manipulation with Pandas

2) Creating DFs from lists

3) Working with dictionaries

4) Data visualization with plotly express

5) Working with dates

6) Changing values in a dataframe using indexing

7) Merging (joining) dataframes in Pandas on multiple columns, and taking care of the necessary preconditions for a successful join

---

### New useful things I've learned:


1) Renaming single columns by their integer index

`df.rename(columns={df.columns[int_index]: 'new_name'},inplace=True)`

2) Using `pd.groupby()` method + `.size()`:

`df.groupby('column_name').size()`

3) Creating simple animations with plotly express

3) Creating geographical scatter plots with `px.scatter_geo`

4) What fuzzy matching is, how to apply it & why it doesn't suit my needs for this project :)

## Imports and preparation

In [1]:
import numpy as np 
import pandas as pd 
import plotly.express as px

In [2]:
df = pd.read_csv("/kaggle/input/layoffs-data-2022/layoffs_data.csv")

## Initial exploration of the data - a first overview :)

In [3]:
## A first look at the data

df.head() ## I don't like that the column names are capitalized. I'll change that in a bit.

## But I can already think of some interesting questions to try and answer with this data

,Company,Location,Industry,Laid_Off_Count,Date,Source,Funds_Raised,Stage,Date_Added,Country,Percentage,List_of_Employees_Laid_Off
0,Vedantu,Bengaluru,Education,385.0,2022-12-07,https://economictimes.indiatimes.com/tech/star...,292.0,Series E,2022-12-07 16:28:02,India,NaN,Unknown
1,Loft,Sao Paulo,Real Estate,312.0,2022-12-07,https://www.bloomberglinea.com/english/brazili...,788.0,Unknown,2022-12-08 06:26:02,Brazil,0.12,Unknown
2,Plaid,SF Bay Area,Finance,260.0,2022-12-07,https://techcrunch.com/2022/12/07/plaid-layoff...,734.0,Series D,2022-12-07 16:22:04,United States,0.20,https://docs.google.com/spreadsheets/d/1Tqw-Iu...
3,Motive,SF Bay Area,Transportation,237.0,2022-12-07,https://gomotive.com/blog/shoaib-makanis-messa...,567.0,Series F,2022-12-08 06:21:12,United States,0.06,Unknown
4,Relativity,Chicago,Legal,150.0,2022-12-07,https://www.chicagobusiness.com/technology/sof...,125.0,Private Equity,2022-12-08 06:22:20,United States,0.10,Unknown


## Questions:

- Top 10 companies with most layoffs per year?
- Top 10 countries with most layoffs per year?
- Top 10 industries with most layoffs per year?
- Map of the world with layoffs per city, per year?

- Average percentage of layoffs per industry, per country?
- Companies with highest layoffs percentages?
- Companies with most layoff "rounds"?

In [4]:
## Let's change column names to all lower case
df.columns = df.columns.str.lower()

df.columns ## Let's see what we've done

Index(['company', 'location', 'industry', 'laid_off_count', 'date', 'source',
       'funds_raised', 'stage', 'date_added', 'country', 'percentage',
       'list_of_employees_laid_off'],
      dtype='object')

In [5]:
## Let's see how the rows are structured:

print(f"Total N of rows: {df.shape[0]}")
print(f"Unique companies: {df.drop_duplicates(subset = ['company']).shape[0]}")
print(f"Unique companies + locations: {df.drop_duplicates(subset = ['company', 'location']).shape[0]}")
print(f"Unique companies + locations + layoff dates: {df.drop_duplicates(subset = ['company', 'location', 'date']).shape[0]}")
print(f"Unique companies + locations + layoff dates + date added: {df.drop_duplicates(subset = ['company', 'location', 'date', 'date_added']).shape[0]}")

Total N of rows: 1757
Unique companies: 1466
Unique companies + locations: 1494
Unique companies + locations + layoff dates: 1749
Unique companies + locations + layoff dates + date added: 1757


In [6]:
def check_nunique_missing(df):
    """
    This is a nice little function I'm borrowing from another Kaggle user and slightly modifying.
    It has all the same data as "pd.DataFrame.info()" method, but also includes:
        - the number of unique values per column and
        - explicit number of NAs per column
    """
    check_list = []
    
    for col in df.columns:
        
        dtypes = df[col].dtypes
        nunique = df[col].nunique()
        not_na = df[col].notna().sum()
        sum_na = df[col].isna().sum()
        
        ## I'm creating a list of lists - with one "sublist" per each column of the original df
        check_list.append([col, dtypes, nunique, not_na, sum_na]) 
        
    df_check = pd.DataFrame(check_list) ## list (of lists) -> pd.DataFrame
    
    df_check.columns = ['column', 'dtypes', 'nunique', 'not_na', 'sum_na'] ## Setting appropriate column names
    
    return df_check 

check_nunique_missing(df)

## So, a lot of NAs in the "laid_off_count" and "percentage" columns

,column,dtypes,nunique,not_na,sum_na
0,company,object,1466,1757,0
1,location,object,162,1757,0
2,industry,object,28,1757,0
3,laid_off_count,float64,235,1237,520
4,date,object,407,1756,1
5,source,object,1584,1757,0
6,funds_raised,float64,561,1627,130
7,stage,object,15,1757,0
8,date_added,object,1718,1757,0
9,country,object,55,1757,0


In [7]:
df['date'] = pd.to_datetime(df['date']) ## Changing format
df['date'].describe(datetime_is_numeric = True) ## What timeframe does the data fall into? ## 2020-2022

count                             1756
mean     2021-10-08 23:25:33.485193728
min                2020-03-11 00:00:00
25%                2020-05-12 00:00:00
50%                2022-06-10 00:00:00
75%                2022-08-31 00:00:00
max                2022-12-07 00:00:00
Name: date, dtype: object

In [8]:
df['year'] = df['date'].dt.year ## Let's extract years from the "date" column, and store them in a separate column
df['year'][:5] ## Let's have a quick look

0    2022.0
1    2022.0
2    2022.0
3    2022.0
4    2022.0
Name: year, dtype: float64

## Companies

In [9]:
## Generating an emty DF to concatenate subequent ones with
companies_data = pd.DataFrame([])

## For each year
for year in [2020, 2021, 2022]:
    
    ## I'm grouping data by "company", summing up total count of layoffs, sorting the results in descending orfer,
    ## limiting the output to only the first 10 rows and resetting index 
    new_data = pd.DataFrame(df[df['year'] == year].groupby(['company'])['laid_off_count'].sum()\
                            .sort_values(ascending = False)\
                            .head(10))\
                            .reset_index()
    ## I'm adding the "year" column to the resulting df
    new_data['year'] = year
    
    ## Concatenating "companies_data" and "new_data"
    ## This should leave us with 10 rows per each of the 3 years [2020, 2021, 2022] -> 10 * 3 = 30 rows of data
    companies_data = pd.concat([companies_data, new_data])

companies_data.reset_index(drop = True, inplace = True)
print(companies_data)
    
fig = px.bar(companies_data, 
             x = 'company', 
             y = 'laid_off_count',  
             animation_frame = "year",
             labels=dict(company = "Companies", laid_off_count = "Total Layoffs"))

max_layoff = companies_data['laid_off_count'].max() + 1000

fig.update_yaxes(range = [0, max_layoff])

fig.show()

          company  laid_off_count  year
0            Uber          7525.0  2020
1     Booking.com          4375.0  2020
2         Groupon          2800.0  2020
3          Swiggy          2250.0  2020
4          Airbnb          1900.0  2020
5           Agoda          1500.0  2020
6     PaisaBazaar          1500.0  2020
7      Stitch Fix          1400.0  2020
8             Ola          1400.0  2020
9           Toast          1300.0  2020
10      Bytedance          3600.0  2021
11        Katerra          2434.0  2021
12         Zillow          2000.0  2021
13      Instacart          1877.0  2021
14    WhiteHat Jr          1800.0  2021
15     Better.com           900.0  2021
16        Dropbox           315.0  2021
17  Delivery Hero           300.0  2021
18        ThredUp           243.0  2021
19         Bounce           200.0  2021
20           Meta         11000.0  2022
21         Amazon         10000.0  2022
22          Cisco          4100.0  2022
23        Peloton          4084.0  2022


## Countries

In [10]:
countries_data = pd.DataFrame([])

for year in [2020, 2021, 2022]:
    
    new_data = pd.DataFrame(df[df['year'] == year].groupby(['country'])['laid_off_count'].sum()\
                            .sort_values(ascending = False)\
                            .head(10))\
                            .reset_index()
    
    new_data['year'] = year
    
    countries_data = pd.concat([countries_data, new_data])

countries_data.reset_index(drop = True, inplace = True)
print(countries_data)

fig = px.bar(countries_data, 
             x = 'country', 
             y = 'laid_off_count',  
             animation_frame = "year",
             labels=dict(country = "Countries", laid_off_count = "Total Layoffs"))

max_layoff = countries_data['laid_off_count'].max() + 1000

fig.update_yaxes(range = [0, max_layoff])

fig.show()

## Here, we can really see how imperfect our dataset is. It is by no means a complete, representative overview
## of layoffs - just have a look at how few layoffs in UK it has recorded

                 country  laid_off_count  year
0          United States         50355.0  2020
1                  India         12932.0  2020
2            Netherlands          4600.0  2020
3                 Brazil          3341.0  2020
4              Singapore          2361.0  2020
5         United Kingdom          1858.0  2020
6                 Canada          1211.0  2020
7              Indonesia           880.0  2020
8              Hong Kong           700.0  2020
9   United Arab Emirates           536.0  2020
10         United States          8670.0  2021
11                 India          4080.0  2021
12                 China          1800.0  2021
13               Germany           387.0  2021
14                Canada            45.0  2021
15             Singapore            21.0  2021
16        United Kingdom            20.0  2021
17         United States        101364.0  2022
18                 India         13974.0  2022
19                Brazil          4496.0  2022
20           

## Industries

In [11]:
industries_data = pd.DataFrame([])

for year in [2020, 2021, 2022]:
    
    new_data = pd.DataFrame(df[df['year'] == year].groupby(['industry'])['laid_off_count'].sum()\
                            .sort_values(ascending = False)\
                            .head(10))\
                            .reset_index()
    
    new_data['year'] = year
    
    industries_data = pd.concat([industries_data, new_data])

industries_data.reset_index(drop = True, inplace = True)
print(industries_data)    

fig = px.bar(industries_data, 
             x = 'industry', 
             y = 'laid_off_count',  
             animation_frame = "year",
             labels=dict(industry = "Industry", laid_off_count = "Total Layoffs"))

max_layoff = industries_data['laid_off_count'].max() + 1000

fig.update_yaxes(range = [0, max_layoff])

fig.show()

          industry  laid_off_count  year
0   Transportation         14656.0  2020
1           Travel         13983.0  2020
2          Finance          8624.0  2020
3           Retail          8002.0  2020
4             Food          6218.0  2020
5         Consumer          6063.0  2020
6      Real Estate          3699.0  2020
7          Fitness          3142.0  2020
8        Marketing          2958.0  2020
9       Recruiting          2382.0  2020
10        Consumer          3600.0  2021
11     Real Estate          2900.0  2021
12            Food          2644.0  2021
13    Construction          2434.0  2021
14       Education          1943.0  2021
15           Other           515.0  2021
16          Retail           288.0  2021
17  Transportation           200.0  2021
18  Infrastructure           160.0  2021
19            Data            90.0  2021
20          Retail         19281.0  2022
21        Consumer         18931.0  2022
22  Transportation         14156.0  2022
23         Finan

## CITIES MAP :)

Now, let's add another dataset to be able to plot cities on a map!

In [12]:
cities_df = pd.read_csv('/kaggle/input/world-cities-database-free-basic/worldcities.csv')

cities_df.head() ## I'm interested in joing the two dataframes on "city_ascii" = "city" and "country" columns

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6839,139.7744,Japan,JP,JPN,Tōkyō,primary,39105000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,35362000.0,1360771077
2,Delhi,Delhi,28.6667,77.2167,India,IN,IND,Delhi,admin,31870000.0,1356872604
3,Manila,Manila,14.6000,120.9833,Philippines,PH,PHL,Manila,primary,23971000.0,1608618140
4,São Paulo,Sao Paulo,-23.5504,-46.6339,Brazil,BR,BRA,São Paulo,admin,22495000.0,1076532519


In [13]:
## Since we'll be using df['location'] and df['country'] columns for the merging, 
## ... let's have a look at values stored there.

print(f"CITIES in the 'df' (layoffs) dataset: \n\n{list(df['location'].unique())}")
print('\n')
print(f"Number of cities in the 'df' (layoffs) dataset: {df['location'].nunique()}")
print("-------")
print(f"COUNTRIES in the 'df' (layoffs) dataset: \n\n{list(df['country'].unique())}")
print('\n')
print(f"Number of countries in the 'df' (layoffs) dataset: {df['country'].nunique()}")

## So, just by eye-balling it, I see we might have problems with joining on the following values:
## "SF Bay Area", "New York City", "Ho Chi Minh City"

## County names look pretty good.
## I won't focus on them too much, even though they are super important for the ...
## ... proper functioning of my join.


CITIES in the 'df' (layoffs) dataset: 

['Bengaluru', 'Sao Paulo', 'SF Bay Area', 'Chicago', 'Stockholm', 'New York City', 'Berlin', 'Los Angeles', 'Tel Aviv', 'London', 'Phoenix', 'Boston', 'Brisbane', 'Lehi', 'Gurugram', 'Singapore', 'Burlington', 'Bend', 'Jakarta', 'Sacramento', 'Buenos Aires', 'Mexico City', 'Melbourne', 'Lagos', 'Amsterdam', 'Guadalajara', 'Waterloo', 'Seattle', 'Sydney', 'Zug', 'Salt Lake City', 'Toronto', 'Cairo', 'Dubai', 'Dallas', 'Washington D.C.', 'Gothenburg', 'Dublin', 'San Diego', 'Calgary', 'Nairobi', 'Dover', 'Austin', 'Hamburg', 'Luxembourg', 'Vancouver', 'Miami', 'Malmö', 'Boulder', 'Lexington', 'Shenzen', 'Barcelona', 'Stamford', 'Logan', 'Tallin', 'Atlanta', 'Eindhoven', 'Columbus', 'Athens', 'Minneapolis', 'Krakow', 'Hyderabad', 'Nebraska City', 'Copenhagen', 'Non-U.S.', 'Trondheim', 'Grand Rapids', 'Oslo', 'Pittsburgh', 'Düsseldorf', 'Montreal', 'Mumbai', 'San Luis Obispo', 'Jerusalem', 'New Delhi', 'Belo Horizonte', 'Baltimore', 'Philadelphia', '

In [14]:
## ["New York City", "SF Bay Area", "Ho Chi Minh City"] from df['location']...

## ... correspond to...

cities_df[cities_df['city_ascii'].str.lower().isin(['new york', 'san francisco', 'ho chi minh'])]

## ["New York", "San Francisco", NA] from cities_df['city_ascii']

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
12,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,18713220.0,1840034016
214,San Francisco,San Francisco,37.7562,-122.4430,United States,US,USA,California,NaN,3592294.0,1840021543
7512,San Francisco,San Francisco,13.7000,-88.1000,El Salvador,SV,SLV,Morazán,admin,21049.0,1222399599
10444,San Francisco,San Francisco,30.9000,-112.6000,Mexico,MX,MEX,Sonora,NaN,89122.0,1484657489
10762,San Francisco,San Francisco,8.5050,125.9771,Philippines,PH,PHL,Agusan del Sur,NaN,80760.0,1608328732
11852,San Francisco,San Francisco,-31.4356,-62.0714,Argentina,AR,ARG,Córdoba,minor,62000.0,1032491643
12019,San Francisco,San Francisco,10.6500,124.3500,Philippines,PH,PHL,Cebu,NaN,59236.0,1608306591
13180,San Francisco,San Francisco,16.8017,-89.9342,Guatemala,GT,GTM,Petén,minor,47919.0,1320240607
28429,San Francisco,San Francisco,15.6667,-87.0333,Honduras,HN,HND,Atlántida,NaN,11206.0,1340403835
33311,San Francisco,San Francisco,4.9711,-74.2892,Colombia,CO,COL,Cundinamarca,minor,8304.0,1170500747


In [15]:
df.replace({'location': 'New York City'}, {'location': 'New York'}, inplace = True)
df.replace({'location': 'SF Bay Area'}, {'location': 'San Francisco'}, inplace = True)

print(len(df[df['location'].isin(['New York City', 'SF Bay Area'])])) ## A quick check

## There's quite a few Washingtons in USA, so this is also a necessary renaming:
cities_df.loc[cities_df["admin_name"] == "District of Columbia", "city_ascii"] = "Washington D.C."

0


In [16]:
## Merging the dfs. Using all lowercase values, to avoid any potential issues due to discrepancies there.

new_df = df.merge(cities_df, how='inner',
                  left_on=[df['country'].str.lower(), df['location'].str.lower()], 
                  right_on=[cities_df['country'].str.lower(), cities_df['city_ascii'].str.lower()])

print(f"Unique cities in the 'df' (layoffs) dataset: {df['location'].nunique()}")
print(f"Unique cities in the 'new_df' (layoffs) dataset: {new_df['location'].nunique()}")
print(f"N of cities lost in the merging: {df['location'].nunique() - new_df['location'].nunique()}")

Unique cities in the 'df' (layoffs) dataset: 162
Unique cities in the 'new_df' (layoffs) dataset: 144
N of cities lost in the merging: 18


Let's see if we can bring this number down a bit.

In [17]:
# initializing list and convert into set object

x = set(df['location'].str.lower().values)
y = set(cities_df['city_ascii'].str.lower().values)

# take difference of two lists
z = x.difference(y)

print(f"City names that are present in df['location'], but not in cities_df['city_ascii']: \n\n{list(z)}")

City names that are present in df['location'], but not in cities_df['city_ascii']: 

['yangon', 'non-u.s.', 'joinville', 'tallin', 'bengaluru', 'quebec', 'tel aviv', 'selangor', 'düsseldorf', 'ferdericton', 'tampa bay', 'shenzen', 'florianópolis', 'noida', 'gurugram', 'malmö']


In [18]:
## Let's see if the above city names have a close match in cities_df['city_ascii']

import difflib

counter = 1

for name in sorted(list(z)):
    
    print(f"{counter}. {name} --> {difflib.get_close_matches(name, cities_df['city_ascii'].str.lower())}")
    
    counter += 1

1. bengaluru --> ['belluru', 'abengourou', 'nguru']
2. düsseldorf --> ['dusseldorf', 'dielsdorf', 'adelsdorf']
3. ferdericton --> ['fredericton', 'frederickson', 'federacion']
4. florianópolis --> ['florianopolis', 'floriano', 'goianapolis']
5. gurugram --> ['gourrama', 'uruara', 'gurupa']
6. joinville --> ['joinvile', 'morinville', 'woodinville']
7. malmö --> ['malmo', 'matalom', 'matalam']
8. noida --> ['noda', 'nida', 'nokia']
9. non-u.s. --> ['onrus', 'nyons']
10. quebec --> ['quebec city', 'ube', 'queven']
11. selangor --> ['shenango', 'erlanger', 'solano']
12. shenzen --> ['shenzhen', 'senden', 'senden']
13. tallin --> ['tallinn', 'stallings', 'tullins']
14. tampa bay --> ['mbamba bay', 'tampa', 'tamba']
15. tel aviv --> ['telavi', 'tel aviv-yafo', 'tela']
16. yangon --> ['langon', 'angono', 'nyaungdon']


In [19]:
old_new_names = {'düsseldorf' : 'Dusseldorf', 
                 'ferdericton' : 'Fredericton', 
                 'florianópolis' : 'Florianopolis',
                 'joinville' : 'Joinvile',      
                 'malmö' : 'malmo',
                 'quebec' : 'quebec city',
                 'shenzen' : 'Shenzhen',
                 'tallin' : 'Tallinn', 
                 'tel aviv' : 'tel aviv-yafo'}

## Replace old name with the new name
for name in old_new_names.keys():
    df.loc[df["location"].str.lower() == name, 'location'] = old_new_names[name]

In [20]:
## Re-do the merging, with the updated names

new_df = df.merge(cities_df, how='inner',
                  left_on=[df['country'].str.lower(), df['location'].str.lower()], 
                  right_on=[cities_df['country'].str.lower(), cities_df['city_ascii'].str.lower()])

print(f"Unique cities in the 'df' (layoffs) dataset: {df['location'].nunique()}")
print(f"Unique cities in the 'new_df' (layoffs) dataset: {new_df['location'].nunique()}")
print(f"N of cities lost in the merging: {df['location'].nunique() - new_df['location'].nunique()}")

Unique cities in the 'df' (layoffs) dataset: 161
Unique cities in the 'new_df' (layoffs) dataset: 152
N of cities lost in the merging: 9


Ok, I'm happy with only 9 cities "lost" in the merging. 

It's the best I can do since, as we've seen above, these remaining 9 cities from `df` don't have a clear match in `cities_df`.

Now, let's make a geo plot :)

In [21]:
new_df['laid_off_count'].fillna(0, inplace=True) # Must not contain NAs, for the px.scatter_geo to work properly

for projection in ['natural earth', 'orthographic']:

    fig = px.scatter_geo(new_df[new_df['country_x'] == 'United States'],
                     lat='lat', ## from cities_df :) This is why we were merging in the first place
                     lon='lng', ## from cities_df :) This is why we were merging in the first place
                     hover_name='location',
                     size="laid_off_count",
                     animation_frame="year",
                     projection=projection,
                     title='Layoffs Tracker')

    fig.show()